In [3]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

import tkinter as tk
from tkinter import filedialog as fd

import collections

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

In [2]:
# open file dialog / catastro de avance
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".shp", ".shp")])
root.destroy()

# open file dialog / xlsx de avance
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()

In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [39]:
file_cat_avance = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/SHP/Cosecha2023.shp'
file_xlsx_avance = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/RENDIMIENTO AVANCE 310723.xlsx'
file_xlsx_estimativa = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/ESTIMATIVAS_DB.xlsx'

In [40]:
catastro_avance = gpd.read_file(file_cat_avance)

In [42]:
data_avance = pd.read_excel(file_xlsx_avance, sheet_name='Hoja1')

In [45]:
# extraer codigos de cosecha
codigos_cosecha = list(set(data_avance['COD COS']))
codigos_cosecha = [int(numero) for numero in codigos_cosecha]

In [46]:
print(codigos_cosecha)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,

In [94]:
dict_keys = {}

In [96]:
for i in codigos_cosecha:
    
    ## GRUPOS
    # seleccion de grupo aguilera
    cod_co = i
    # filtrado por codigo de cosecha
    grupo = data_avance[data_avance['COD COS'] == cod_co]
    # seleccion de columnas de interes
    grupo = grupo[['CODIGO CAÑERO ', 'COD COS', 'CAÑERO', 'TN COMPROMISO INDIVIDUAL', 'TN ENTREGA INDIVIDUAL']]
    # calculo de % de avance de entrega del grupo
    grupo['avance_porcen'] = (grupo['TN ENTREGA INDIVIDUAL'] / grupo['TN COMPROMISO INDIVIDUAL'])*100
    # cambio del nombre del campo codigo cañero
    grupo = grupo.rename(columns={'CODIGO CAÑERO ': 'cod_ca'})
    list_obj_grupo = []
    for index, row in grupo.iterrows():
        canero = {
            'cod_ca' : row['cod_ca'],
            'nom_ca' : row['CAÑERO'],
            'compromiso' : row['TN COMPROMISO INDIVIDUAL'],
            'entrega' : row['TN ENTREGA INDIVIDUAL'],
            'porcen' : row['avance_porcen']
        }
        list_obj_grupo.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
    # TOTALES DE GRUPO
    # totales de tabla de entreas del grupo seleccionado
    grupo_totales = {
        'compromiso':grupo['TN COMPROMISO INDIVIDUAL'].sum(),
        'entrega':grupo['TN ENTREGA INDIVIDUAL'].sum(),
        'porcen':grupo['TN ENTREGA INDIVIDUAL'].sum() / grupo['TN COMPROMISO INDIVIDUAL'].sum() * 100
    }
    obj_grupo_totales = collections.namedtuple("totales", grupo_totales.keys())(*grupo_totales.values())
    
    # AVANCE POR PROPIEDAD
    xlsx_estimativas = pd.read_excel(file_xlsx_estimativa, sheet_name='ESTIMATIVA_PROPS')
    xlsx_estimativas = xlsx_estimativas.rename(columns={'CODIGO' : 'unidad_01'})
    hoy = datetime.now()
    mes_actual = hoy.month
    mes_actual
    # cambio de nombre de columna para hacer merge con la tabla de grupo
    catastro_avance = catastro_avance.rename(columns={'unidad_03':'cod_ca'})
    # merge 
    catastro_merge = pd.merge(grupo, catastro_avance, on='cod_ca')
    
    if len(catastro_merge)==0:
        continue
    
    # crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
    catastro_merge['area_cosechada'] = catastro_merge.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
    # calculo de area cosechada en el mes actual
    catastro_merge['avance_mes'] = catastro_merge.apply(lambda row : row['area'] if row['fc']!=None and row['fc'].split('-')[1] == str(mes_actual) else 0, axis=1)
    # agrupacion de datos por pripiedad, resumiendo por area, area_cosechada, avance_mes
    propiedades = catastro_merge.groupby(['unidad_01', 'unidad_02'])[['area', 'area_cosechada', 'avance_mes']].sum()
    # reseteo de index
    propiedades = propiedades.reset_index()
    # calculo de % de avance de cosecha
    propiedades['%avance'] = propiedades['area_cosechada'] / propiedades['area'] * 100
    # merge con las estimativas de produccion de cada propiedad
    propiedades = pd.merge(propiedades, xlsx_estimativas, on='unidad_01')
    # seleccion de columnas de interes
    propiedad = propiedades[['unidad_01', 'unidad_02', 'area', 'area_cosechada', '%avance', 'PRODUCCION']]
    
    list_obj_props = []
    for index, row in propiedad.iterrows():
        prop = {
            'cod_prop' : row['unidad_01'],
            'nom_prop' : row['unidad_02'],
            'area' : row['area'],
            'area_cosechada' : row['area_cosechada'],
            'avance_porcen' : row['%avance'],
            'produccion' : row['PRODUCCION']
        }
        list_obj_props.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))
    
    # totales de tabla por columna
    propiedades_totales = {
        'area':propiedades['area'].sum(),
        'area_cosechada':propiedades['area_cosechada'].sum(),
        'avance_mes':propiedades['avance_mes'].sum(),
        'avance_porcen':propiedades['area_cosechada'].sum() / propiedades['area'].sum() * 100,
        'produccion' : propiedad['PRODUCCION'].sum()
    }
    propiedades_totales
    obj_props_totales = collections.namedtuple("total", propiedades_totales.keys())(*propiedades_totales.values())
    
    # AVANCE POR VARIEDAD
    # merge 
    catastro_merge_2 = pd.merge(grupo, catastro_avance, on='cod_ca')
    # crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
    catastro_merge_2['area_cosechada'] = catastro_merge_2.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
    # extraer valores unicos en el campo unidad_02
    nom_props = list(set(catastro_merge_2['unidad_02']))
    # lista de los codigos de propiedades
    cod_props = list(set(catastro_merge_2['unidad_01'])) ##########################################################################################################
    # se crea un dict que relacian al grupo con las propiedades
    dict_keys[i] = cod_props
    
    # lista de objetos para almacenar una lista de propiedades
    list_obj_props_var = []
    for prop in nom_props:
        # dict para almacenar una propiedad en concreto y su avance por variedad
        data_obj_props = {}
        data_obj_props['prop'] = prop
        # selecciona todos los datos de una priedad en concreto segun en nombre en el campo unidad_02
        avance_variedad = catastro_merge_2[catastro_merge_2['unidad_02']==prop]
        # agrupa los datos segun variedad
        avance_variedad = avance_variedad.groupby(['variedad'])[['area_cosechada', 'area']].sum()
        # reinicia el index de dataframe
        avance_variedad = avance_variedad.reset_index()
        # ordemar por area cosechadad
        avance_variedad = avance_variedad.sort_values(['area_cosechada'], ascending=False)
        # calculo de porcentaje de avance
        avance_variedad['porcen_avan'] = avance_variedad.apply(lambda row : row['area_cosechada']/row['area'] *100, axis=1)
        # suma de totales
        totales_variedad = {
            'total_cosechado' : avance_variedad['area_cosechada'].sum(),
            'total_area' : avance_variedad['area'].sum(),
            'total_porcen' : avance_variedad['area_cosechada'].sum() / avance_variedad['area'].sum() * 100
        }
        # add totales a dict de datos de la prop
        data_obj_props['totales'] = collections.namedtuple("totales", totales_variedad.keys())(*totales_variedad.values())
    
        # objeto para almacenar las variedades
        list_obj_vars = []
        # recorres el dataframe de avande por variedad para convertir en obj cada fila
        for index, row in avance_variedad.iterrows():
            var = {
                'var':row['variedad'],
                'area_cosecha':row['area_cosechada'],
                'area':row['area'],
                'porcen_avan':row['porcen_avan']
            }
            list_obj_vars.append(collections.namedtuple("variedad", var.keys())(*var.values()))
        data_obj_props['variedades'] = list_obj_vars
        list_obj_props_var.append(collections.namedtuple("propiedad", data_obj_props.keys())(*data_obj_props.values()))
    
    ## AVANCE POR CORTE
    # merge 
    catastro_merge_3 = pd.merge(grupo, catastro_avance, on='cod_ca')
    # crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
    catastro_merge_3['area_cosechada'] = catastro_merge_3.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
    # extraer valores unicos en el campo unidad_02
    nom_props = list(set(catastro_merge_3['unidad_02']))
    # lista de objetos para almacenar una lista de propiedades
    list_obj_props_soca = []
    for prop in nom_props:
        # dict para almacenar una propiedad en concreto y su avance por variedad
        data_obj_props = {}
        data_obj_props['prop'] = prop
        avance_corte = catastro_merge_3[catastro_merge_3['unidad_02']==prop]
        #avance_corte['soca'] = avance_corte['soca'].astype(int)
        suma_area = avance_corte.loc[avance_corte['soca'] >= 10, 'area'].sum()
        if (suma_area > 0):
            suma_area_cosechada = avance_corte.loc[avance_corte['soca'] >= 10, 'area_cosechada'].sum()
            avance_corte= avance_corte[avance_corte['soca'] < 10]
            avance_corte['soca'] = avance_corte['soca'].astype(str)
            new_reg = {'soca': '>10', 'area': suma_area, 'area_cosechada': suma_area_cosechada}
            avance_corte = avance_corte.append(new_reg, ignore_index=True)
        else:
            avance_corte['soca'] = avance_corte['soca'].astype(str)
        #agrupacion de datos por propiedad, resumiendo por area, area_cosechada, avance_mes
        avance_corte = avance_corte.groupby(['soca'])[['area_cosechada', 'area']].sum()
        # reseteo de index
        avance_corte = avance_corte.reset_index()
        # ordemar por soca
        avance_corte = avance_corte.sort_values(['soca'])
        # calculo de porcentaje de avance
        avance_corte['porcen_avan'] = avance_corte.apply(lambda row : row['area_cosechada']/row['area'] *100, axis=1)
    
    
        list_obj_soca = []
        # recorres el dataframe de avance por soca para convertir en obj cada fila
        for index, row in avance_corte.iterrows():
            soca = {
                'soca': row['soca'],
                'area_cosecha':row['area_cosechada'],
                'area':row['area'],
                'porcen_avan':row['porcen_avan']
            }
            list_obj_soca.append(collections.namedtuple("soca", soca.keys())(*soca.values()))
        data_obj_props['socas'] = list_obj_soca
        list_obj_props_soca.append(collections.namedtuple("propiedad", data_obj_props.keys())(*data_obj_props.values()))
    
    ## AVANCE GENERAL
    fecha_anterior = datetime(2023, 5, 5)
    fecha_actual = datetime.now()
    dias_zafra = (fecha_actual - fecha_anterior).days
    totales_avance_gral = {
        'avance_area_porcen' : propiedades_totales['avance_porcen'],
        'avance_tn_porcen' : grupo_totales['porcen'],
        'tiempo_porcen' : (dias_zafra / 171) * 100,
        'area_requerida' : propiedades_totales['area'] * (dias_zafra / 171),
        'cumplimiento_area_porcen' : propiedades_totales['area_cosechada']/(propiedades_totales['area'] * (dias_zafra / 171)) * 100,
        'tn_requeridas' : (propiedades_totales['produccion'])*(dias_zafra / 171),
        'cumplimiento_tn_porcen' : grupo_totales['entrega']/((propiedades_totales['produccion'])*(dias_zafra / 171)) * 100
    }
    totales_avance_gral
    obj_avance_gral = collections.namedtuple("avance_gral", totales_avance_gral.keys())(*totales_avance_gral.values())
    
    doc = DocxTemplate(ruta + "/_test/tpl_rpt_avance_cosecha.docx")
    context = {
        'grupo' : list_obj_grupo, 
        'g_totales' : obj_grupo_totales,
        'prop' : list_obj_props,
        'p_totales' : obj_props_totales,
        'vars' : list_obj_props_var,
        'soca' : list_obj_props_soca,
        'avance' : obj_avance_gral
    }
    doc.render(context)
    doc.save(ruta + '/_test/' + str(i) + '.docx')
    print('Se generó reporte de grupo:', str(i))
print('END')

END


In [99]:
dict_keys

{1: [1696, 17, 1748],
 2: [265, 369, 211, 1816, 1817],
 3: [1820, 1960, 749, 750, 1818, 1819, 732, 733, 1694],
 4: [787],
 5: [1, 218, 19],
 6: [18],
 7: [949],
 8: [664],
 9: [74],
 10: [258],
 11: [47],
 12: [600, 111],
 13: [1635, 1315, 1726, 1585, 1310],
 14: [979, 981, 982],
 15: [1464, 1592, 788],
 16: [941],
 17: [1225, 1010, 761, 1467, 796],
 18: [1594, 519],
 19: [791],
 20: [520, 1088],
 21: [903, 1295, 1525, 1718, 925],
 22: [789, 790, 792, 793, 794],
 23: [333],
 24: [1774, 1855],
 25: [766],
 26: [320],
 28: [1100, 277],
 29: [1656, 1961, 1962],
 30: [216, 256, 217],
 31: [197, 198, 969, 1450, 877, 84, 1049, 700, 1213, 1214],
 32: [930, 1287, 1171, 186, 1244, 927],
 33: [325],
 34: [1336, 1337, 1645, 1335],
 35: [288],
 36: [1615],
 37: [327],
 38: [141, 142],
 39: [285, 286, 287],
 40: [629],
 41: [30],
 42: [115],
 43: [1015],
 44: [1561, 1090],
 45: [1833, 551],
 46: [1618, 1707],
 47: [343],
 48: [559],
 49: [1442],
 50: [122],
 51: [257],
 52: [219, 324],
 54: [928, 9

## COMPROMISO Y ENTREGAS

In [48]:
# seleccion de grupo aguilera
cod_co = 108
# filtrado por codigo de cosecha
grupo = data_avance[data_avance['COD COS'] == cod_co]
# seleccion de columnas de interes
grupo = grupo[['CODIGO CAÑERO ', 'COD COS', 'CAÑERO', 'TN COMPROMISO INDIVIDUAL', 'TN ENTREGA INDIVIDUAL']]
# calculo de % de avance de entrega del grupo
grupo['avance_porcen'] = (grupo['TN ENTREGA INDIVIDUAL'] / grupo['TN COMPROMISO INDIVIDUAL'])*100
# cambio del nombre del campo codigo cañero
grupo = grupo.rename(columns={'CODIGO CAÑERO ': 'cod_ca'})
grupo

,cod_ca,COD COS,CAÑERO,TN COMPROMISO INDIVIDUAL,TN ENTREGA INDIVIDUAL,avance_porcen
298,2045,108,COLQUE LIZARAZO ISABEL,2091.92,1201.96,57.457264


In [49]:
list_obj_grupo = []
for index, row in grupo.iterrows():
    canero = {
        'cod_ca' : row['cod_ca'],
        'nom_ca' : row['CAÑERO'],
        'compromiso' : row['TN COMPROMISO INDIVIDUAL'],
        'entrega' : row['TN ENTREGA INDIVIDUAL'],
        'porcen' : row['avance_porcen']
    }
    list_obj_grupo.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
list_obj_grupo

[canero(cod_ca=2045, nom_ca='COLQUE LIZARAZO ISABEL', compromiso=2091.92, entrega=1201.96, porcen=57.45726414012008)]

In [50]:
# totales de tabla de entreas del grupo seleccionado
grupo_totales = {
    'compromiso':grupo['TN COMPROMISO INDIVIDUAL'].sum(),
    'entrega':grupo['TN ENTREGA INDIVIDUAL'].sum(),
    'porcen':grupo['TN ENTREGA INDIVIDUAL'].sum() / grupo['TN COMPROMISO INDIVIDUAL'].sum() * 100
}
grupo_totales
obj_grupo_totales = collections.namedtuple("totales", grupo_totales.keys())(*grupo_totales.values())
obj_grupo_totales

totales(compromiso=2091.92, entrega=1201.96, porcen=57.45726414012008)

## AVANCE PROPIEDAD

In [51]:
xlsx_estimativas = pd.read_excel(file_xlsx_estimativa, sheet_name='ESTIMATIVA_PROPS')
xlsx_estimativas = xlsx_estimativas.rename(columns={'CODIGO' : 'unidad_01'})

In [52]:
hoy = datetime.now()
mes_actual = hoy.month
mes_actual

8

In [53]:
# cambio de nombre de columna para hacer merge con la tabla de grupo
catastro_avance = catastro_avance.rename(columns={'unidad_03':'cod_ca'})
# merge 
catastro_merge = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge['area_cosechada'] = catastro_merge.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# calculo de area cosechada en el mes actual
catastro_merge['avance_mes'] = catastro_merge.apply(lambda row : row['area'] if row['fc']!=None and row['fc'].split('-')[1] == str(mes_actual) else 0, axis=1)
# agrupacion de datos por pripiedad, resumiendo por area, area_cosechada, avance_mes
propiedades = catastro_merge.groupby(['unidad_01', 'unidad_02'])[['area', 'area_cosechada', 'avance_mes']].sum()
# reseteo de index
propiedades = propiedades.reset_index()
# calculo de % de avance de cosecha
propiedades['%avance'] = propiedades['area_cosechada'] / propiedades['area'] * 100
# merge con las estimativas de produccion de cada propiedad
propiedades = pd.merge(propiedades, xlsx_estimativas, on='unidad_01')
# seleccion de columnas de interes
propiedad = propiedades[['unidad_01', 'unidad_02', 'area', 'area_cosechada', '%avance', 'PRODUCCION']]
propiedad

ValueError: Wrong number of items passed 23, placement implies 1

In [37]:
list_obj_props = []
for index, row in propiedad.iterrows():
    prop = {
        'cod_prop' : row['unidad_01'],
        'nom_prop' : row['unidad_02'],
        'area' : row['area'],
        'area_cosechada' : row['area_cosechada'],
        'avance_porcen' : row['%avance'],
        'produccion' : row['PRODUCCION']
    }
    list_obj_props.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))
list_obj_props

[propiedad(cod_prop=216, nom_prop='LA_ESTACADA--AGUILERA', area=225.34, area_cosechada=189.72, avance_porcen=84.19277536167569, produccion=15287.860465384705),
 propiedad(cod_prop=217, nom_prop='FLAMBOYAN--AGUILERA', area=207.45999999999998, area_cosechada=23.41, avance_porcen=11.284102959606672, produccion=14105.979992834908),
 propiedad(cod_prop=256, nom_prop='LOS_ANDES--AGUILERA', area=423.24, area_cosechada=218.45, avance_porcen=51.61374161232397, produccion=33770.21854870834)]

In [38]:
# totales de tabla por columna
propiedades_totales = {
    'area':propiedades['area'].sum(),
    'area_cosechada':propiedades['area_cosechada'].sum(),
    'avance_mes':propiedades['avance_mes'].sum(),
    'avance_porcen':propiedades['area_cosechada'].sum() / propiedades['area'].sum() * 100,
    'produccion' : propiedad['PRODUCCION'].sum()
}
propiedades_totales
obj_props_totales = collections.namedtuple("total", propiedades_totales.keys())(*propiedades_totales.values())
obj_props_totales

total(area=856.04, area_cosechada=431.58, avance_mes=0, avance_porcen=50.415868417363676, produccion=63164.05900692796)

# AVANCE POR VARIEDAD

In [39]:
# merge 
catastro_merge_2 = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge_2['area_cosechada'] = catastro_merge_2.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# extraer valores unicos en el campo unidad_02
nom_props = list(set(catastro_merge_2['unidad_02']))
# lista de objetos para almacenar una lista de propiedades
list_obj_props_var = []
for prop in nom_props:
    # dict para almacenar una propiedad en concreto y su avance por variedad
    data_obj_props = {}
    data_obj_props['prop'] = prop
    # selecciona todos los datos de una priedad en concreto segun en nombre en el campo unidad_02
    avance_variedad = catastro_merge_2[catastro_merge_2['unidad_02']==prop]
    # agrupa los datos segun variedad
    avance_variedad = avance_variedad.groupby(['variedad'])[['area_cosechada', 'area']].sum()
    # reinicia el index de dataframe
    avance_variedad = avance_variedad.reset_index()
    # ordemar por area cosechadad
    avance_variedad = avance_variedad.sort_values(['area_cosechada'], ascending=False)
    # calculo de porcentaje de avance
    avance_variedad['porcen_avan'] = avance_variedad.apply(lambda row : row['area_cosechada']/row['area'] *100, axis=1)
    # suma de totales
    totales_variedad = {
        'total_cosechado' : avance_variedad['area_cosechada'].sum(),
        'total_area' : avance_variedad['area'].sum(),
        'total_porcen' : avance_variedad['area_cosechada'].sum() / avance_variedad['area'].sum() * 100
    }
    # add totales a dict de datos de la prop
    data_obj_props['totales'] = collections.namedtuple("totales", totales_variedad.keys())(*totales_variedad.values())
    
    # objeto para almacenar las variedades
    list_obj_vars = []
    # recorres el dataframe de avande por variedad para convertir en obj cada fila
    for index, row in avance_variedad.iterrows():
        var = {
            'var':row['variedad'],
            'area_cosecha':row['area_cosechada'],
            'area':row['area'],
            'porcen_avan':row['porcen_avan']
        }
        list_obj_vars.append(collections.namedtuple("variedad", var.keys())(*var.values()))
    data_obj_props['variedades'] = list_obj_vars
    list_obj_props_var.append(collections.namedtuple("propiedad", data_obj_props.keys())(*data_obj_props.values()))
list_obj_props_var

[propiedad(prop='LA_ESTACADA--AGUILERA', totales=totales(total_cosechado=189.72, total_area=225.33999999999997, total_porcen=84.1927753616757), variedades=[variedad(var='UCG_90-20', area_cosecha=96.41, area=104.64, porcen_avan=92.13493883792049), variedad(var='RB-2', area_cosecha=64.6, area=69.25, porcen_avan=93.28519855595667), variedad(var='CITTCA_85-22', area_cosecha=28.71, area=32.47, porcen_avan=88.42008007391439), variedad(var='SP_83-5073', area_cosecha=0.0, area=18.98, porcen_avan=0.0)]),
 propiedad(prop='LOS_ANDES--AGUILERA', totales=totales(total_cosechado=218.45000000000002, total_area=423.24, total_porcen=51.613741612323984), variedades=[variedad(var='UCG_90-20', area_cosecha=142.33, area=279.63, porcen_avan=50.89940278224797), variedad(var='CITTCA_85-22', area_cosecha=48.77, area=60.370000000000005, porcen_avan=80.78515819115455), variedad(var='SP_83-5073', area_cosecha=14.22, area=21.85, porcen_avan=65.08009153318078), variedad(var='RB-2', area_cosecha=11.450000000000001, 

# AVANCE POR CORTE

In [ ]:
# merge 
catastro_merge_3 = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge_3['area_cosechada'] = catastro_merge_3.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# extraer valores unicos en el campo unidad_02
nom_props = list(set(catastro_merge_3['unidad_02']))
# lista de objetos para almacenar una lista de propiedades
list_obj_props_soca = []
for prop in nom_props:
    # dict para almacenar una propiedad en concreto y su avance por variedad
    data_obj_props = {}
    data_obj_props['prop'] = prop
    avance_corte = catastro_merge_3[catastro_merge_3['unidad_02']==prop]
    #avance_corte['soca'] = avance_corte['soca'].astype(int)
    suma_area = avance_corte.loc[avance_corte['soca'] >= 10, 'area'].sum()
    if (suma_area > 0):
        suma_area_cosechada = avance_corte.loc[avance_corte['soca'] >= 10, 'area_cosechada'].sum()
        avance_corte= avance_corte[avance_corte['soca'] < 10]
        avance_corte['soca'] = avance_corte['soca'].astype(str)
        new_reg = {'soca': '>10', 'area': suma_area, 'area_cosechada': suma_area_cosechada}
        avance_corte = avance_corte.append(new_reg, ignore_index=True)
    else:
        avance_corte['soca'] = avance_corte['soca'].astype(str)
    #agrupacion de datos por propiedad, resumiendo por area, area_cosechada, avance_mes
    avance_corte = avance_corte.groupby(['soca'])[['area_cosechada', 'area']].sum()
    # reseteo de index
    avance_corte = avance_corte.reset_index()
    # ordemar por soca
    avance_corte = avance_corte.sort_values(['soca'])
    # calculo de porcentaje de avance
    avance_corte['porcen_avan'] = avance_corte.apply(lambda row : row['area_cosechada']/row['area'] *100, axis=1)
    
    
    list_obj_soca = []
    # recorres el dataframe de avance por soca para convertir en obj cada fila
    for index, row in avance_corte.iterrows():
        soca = {
            'soca': row['soca'],
            'area_cosecha':row['area_cosechada'],
            'area':row['area'],
            'porcen_avan':row['porcen_avan']
        }
        list_obj_soca.append(collections.namedtuple("soca", soca.keys())(*soca.values()))
    data_obj_props['socas'] = list_obj_soca
    list_obj_props_soca.append(collections.namedtuple("propiedad", data_obj_props.keys())(*data_obj_props.values()))
list_obj_props_soca

## TABLA AVANCE GENERAL

In [42]:
fecha_anterior = datetime(2023, 5, 5)
fecha_actual = datetime.now()
dias_zafra = (fecha_actual - fecha_anterior).days

In [44]:
totales_avance_gral = {
    'avance_area_porcen' : propiedades_totales['avance_porcen'],
    'avance_tn_porcen' : grupo_totales['porcen'],
    'tiempo_porcen' : (dias_zafra / 171) * 100,
    'area_requerida' : propiedades_totales['area'] * (dias_zafra / 171),
    'cumplimiento_area_porcen' : propiedades_totales['area_cosechada']/(propiedades_totales['area'] * (dias_zafra / 171)) * 100,
    'tn_requeridas' : (propiedades_totales['produccion'])*(dias_zafra / 171),
    'cumplimiento_tn_porcen' : grupo_totales['entrega']/((propiedades_totales['produccion'])*(dias_zafra / 171)) * 100
}
totales_avance_gral
obj_avance_gral = collections.namedtuple("avance_gral", totales_avance_gral.keys())(*totales_avance_gral.values())
obj_avance_gral

avance_gral(avance_area_porcen=50.415868417363676, avance_tn_porcen=63.15688378601054, tiempo_porcen=52.046783625730995, area_requerida=445.5412865497076, cumplimiento_area_porcen=96.86644381313695, tn_requeridas=32874.86112056484, cumplimiento_tn_porcen=125.82118552015744)

In [45]:
doc = DocxTemplate(ruta + "/_test/tpl_rpt_avance_cosecha.docx")
context = {
    'grupo' : list_obj_grupo, 
    'g_totales' : obj_grupo_totales,
    'prop' : list_obj_props,
    'p_totales' : obj_props_totales,
    'vars' : list_obj_props_var,
    'soca' : list_obj_props_soca,
    'avance' : obj_avance_gral
}
doc.render(context)
doc.save(ruta + '/_test/data.docx')

# Convertir a PDF

In [59]:
import os

In [60]:
from docx2pdf import convert

In [62]:
path = ruta + '/_test/'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')

SE ENCONTRARON 443 INFORMES
443 INFORMES


In [68]:
ruta

'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [74]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').split('/')[-1]
    convert(filename, r'C:/Users/Bismar/Downloads/' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//1.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//1.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//1.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//10.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//10.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//10.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//100.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//100.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//100.pdf


...

CONVIRTIENDO... INFORME 442 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//99.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//99.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//99.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 443 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//tpl_rpt_avance_cosecha.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//tpl_rpt_avance_cosecha.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/_test//tpl_rpt_avance_cosecha.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


# leer ruta de planos

In [109]:
path = r'D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\_tes_pdf/'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')


SE ENCONTRARON 441 INFORMES
441 INFORMES


In [78]:
reporte = rutas[0]

In [79]:
reporte

'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON\\_tes_pdf//10.pdf'

In [80]:
list_pdfs = []
list_pdfs.append(reporte)

In [81]:
list_pdfs

['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON\\_tes_pdf//10.pdf']

In [93]:
path = r'D:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2023\ANALISIS DE CAÑA EN CAMPO 2023\REPORTE_AVANCE_COSECHA\PLANOS_2'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')

SE ENCONTRARON 1590 INFORMES
1590 INFORMES


In [84]:
data  = {}
for i in rutas:
    cod_prop = i.split('_')[-1].split('.')[0]
    data[cod_prop]=i

In [87]:
data['30']

'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS_2/PLANO_30.pdf'

In [88]:
df

NameError: name 'df' is not defined

In [89]:
for row in df.itertuples():
    cod_prop = row.unidad_01
    ruta_prop = data[str(cod_prop)]
    list_pdfs.append(ruta_prop)

NameError: name 'df' is not defined

In [90]:
list_pdfs

['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON\\_tes_pdf//10.pdf']

ModuleNotFoundError: No module named 'PyPDF2'

In [92]:
filename.replace('reportes', 'pdf_final').replace('docx', 'pdf')

'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/_test//tpl_rpt_avance_cosecha.pdf'

In [35]:
nombre_archivo_salida = filename.replace('reportes', 'pdf_final').replace('docx', 'pdf')

NameError: name 'filename' is not defined

In [ ]:
path = r'D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON\_tes_pdf/'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')

In [1]:
dict_keys = {1: [1696, 17, 1748],
 2: [265, 369, 211, 1816, 1817],
 3: [1820, 1960, 749, 750, 1818, 1819, 732, 733, 1694],
 4: [787],
 5: [1, 218, 19],
 6: [18],
 7: [949],
 8: [664],
 9: [74],
 10: [258],
 11: [47],
 12: [600, 111],
 13: [1635, 1315, 1726, 1585, 1310],
 14: [979, 981, 982],
 15: [1464, 1592, 788],
 16: [941],
 17: [1225, 1010, 761, 1467, 796],
 18: [1594, 519],
 19: [791],
 20: [520, 1088],
 21: [903, 1295, 1525, 1718, 925],
 22: [789, 790, 792, 793, 794],
 23: [333],
 24: [1774, 1855],
 25: [766],
 26: [320],
 28: [1100, 277],
 29: [1656, 1961, 1962],
 30: [216, 256, 217],
 31: [197, 198, 969, 1450, 877, 84, 1049, 700, 1213, 1214],
 32: [930, 1287, 1171, 186, 1244, 927],
 33: [325],
 34: [1336, 1337, 1645, 1335],
 35: [288],
 36: [1615],
 37: [327],
 38: [141, 142],
 39: [285, 286, 287],
 40: [629],
 41: [30],
 42: [115],
 43: [1015],
 44: [1561, 1090],
 45: [1833, 551],
 46: [1618, 1707],
 47: [343],
 48: [559],
 49: [1442],
 50: [122],
 51: [257],
 52: [219, 324],
 54: [928, 932, 361, 1358, 1359],
 55: [1024, 1025, 963, 964, 1700, 1033, 1034],
 56: [554, 398, 720, 755, 757],
 57: [1665, 1896, 623, 756, 1176, 1177, 1178, 1179, 1181],
 58: [1379, 1354, 1364, 1686, 1687, 729, 1342, 159],
 59: [1528, 858, 699],
 60: [301, 302, 303, 304, 305],
 61: [1765, 1324, 398, 1790, 57, 988, 1085, 1086],
 62: [902],
 63: [774, 775],
 64: [1428, 1413],
 65: [1609, 873],
 67: [776, 1321, 46, 719],
 68: [1413],
 69: [416, 417, 418, 419, 1130, 851],
 70: [800, 89, 615, 799],
 71: [833, 1591, 280, 281, 282, 1756, 698],
 72: [459],
 73: [1536, 161, 1537, 1548, 527, 944, 529, 945, 1812],
 74: [162, 1660, 1661],
 75: [1846],
 76: [98, 483, 100, 971, 1550, 313],
 77: [1152],
 78: [133, 783, 80, 313, 1723, 1597, 1598],
 79: [1644, 433, 1144, 1369, 1370],
 80: [1386, 109, 149, 1375],
 81: [1371, 1083, 135],
 82: [1418, 1419, 1420, 227],
 83: [289, 1538, 168, 1385, 1072, 1073, 146, 535],
 84: [1760, 1761, 1762, 1111, 1759],
 85: [730],
 86: [20],
 87: [1040, 1043, 1039],
 88: [934, 1054],
 89: [1008, 1011],
 90: [1611, 1934, 1935, 209, 437, 439, 440, 441, 61, 1695],
 91: [1046],
 92: [1376, 1377, 1378, 1092, 1038, 1042, 83, 1372, 1373, 1374],
 93: [1507, 1508, 44, 1426, 1531],
 94: [1792, 1859, 1860, 1924, 683, 915, 1787],
 95: [1382, 1380, 1350],
 96: [1900, 1901, 1902, 1904, 1905, 1906, 1907, 1908, 1909, 1620, 1621, 1022],
 97: [1066, 1067, 1381],
 98: [627],
 99: [1572, 1384, 1064, 973, 14],
 100: [1417],
 101: [1425, 1755],
 102: [1787],
 103: [1442, 1923, 1174],
 104: [1107, 1404, 1069, 1070],
 105: [816, 1026, 860, 815],
 106: [1282, 1668, 1669, 813, 816],
 107: [1147, 1139, 1140, 1047],
 109: [450, 1379, 1137, 1299, 1301, 444],
 110: [1128, 1122, 1123, 1124],
 111: [936],
 112: [1410, 445],
 113: [1627, 1023],
 114: [1772, 1776, 1299, 435, 1301],
 115: [67, 68, 1095, 1808, 1399, 445, 414],
 116: [521, 447],
 117: [40, 1498],
 118: [1440, 1701, 1554, 1365, 1724],
 119: [1731],
 120: [156, 1628],
 121: [691, 692, 693],
 122: [176, 1659, 1654],
 124: [1294],
 125: [478, 991],
 126: [16, 85, 1806, 1799],
 129: [1408, 28],
 130: [1616, 1617],
 131: [1006],
 132: [1706],
 134: [1267],
 135: [1009],
 136: [613],
 137: [113, 499, 502, 751],
 138: [225, 226, 1325, 500, 501, 374, 1149],
 139: [290, 291, 292, 293, 294, 295, 675],
 140: [309, 310, 239],
 141: [172, 342],
 142: [10, 1423, 1968, 1969, 311, 1972, 1015, 253],
 143: [1771, 996],
 144: [1307, 1306, 995, 1308],
 145: [1409, 1797, 1303, 1304, 1403, 1405, 1406, 1407],
 146: [1573, 458, 1002, 1688, 249, 250, 251],
 147: [1216, 1730, 1764, 1671, 171, 1228, 1325, 1273],
 148: [1830, 1831],
 149: [1411, 527, 1607],
 150: [1250, 1252, 1733, 1254, 1258, 1910, 1880, 1305, 1885, 1919],
 151: [1412],
 152: [944, 1622, 1623, 1629, 1630, 1535],
 153: [576],
 154: [836, 1767],
 155: [958],
 156: [1028, 1036],
 157: [966, 839, 838, 840, 844, 845, 846],
 158: [1633],
 159: [1546],
 161: [1231],
 162: [486, 487, 488, 489, 490, 493, 494, 496],
 163: [1184, 1180, 1182, 1183],
 164: [1895, 872, 1948, 1949, 1950],
 165: [1568],
 166: [236, 237, 238, 1735],
 167: [48, 989],
 168: [573],
 169: [1750, 1751],
 170: [274, 1100, 277],
 171: [231],
 172: [1215, 495, 173, 279],
 173: [32, 33, 31],
 174: [298, 332, 1709],
 175: [1289, 221, 222],
 176: [355, 1805, 1438],
 177: [578, 549, 550, 391, 392],
 179: [1314],
 180: [555],
 181: [174],
 182: [36, 1367],
 183: [2, 66, 1109, 38],
 184: [961, 759],
 185: [1159, 1770, 916, 886, 887, 888, 889, 890, 891, 892, 893],
 186: [1472, 484, 485, 1478, 753, 1684],
 187: [1569, 1154, 1157, 454, 1158, 685, 687, 1170, 690],
 188: [672, 1868, 654, 655, 667, 670, 671],
 189: [1544, 1473, 682],
 190: [1156, 710],
 191: [389, 936, 681, 1872, 1939, 1940],
 192: [834, 835, 854, 855],
 193: [1793],
 194: [328, 53, 13, 55],
 195: [29],
 196: [423, 558, 306, 307, 1491],
 197: [259],
 198: [376, 1474, 375],
 199: [1454, 1619, 345, 27, 669],
 201: [864,
  1505,
  1570,
  1443,
  1190,
  234,
  235,
  1165,
  206,
  1712,
  1778,
  1433,
  1243,
  862],
 202: [803, 1285, 1286, 233, 478],
 203: [275, 132],
 204: [1587, 628, 734],
 205: [5],
 206: [705, 870, 1414, 779, 780, 1198, 1422, 1111, 126],
 207: [1588],
 208: [935, 912, 922, 923, 924],
 209: [273, 714, 118],
 211: [187],
 212: [729],
 213: [1290, 1580],
 214: [1875, 1876, 1566, 1567],
 215: [1952, 42, 1291, 1180, 858, 1275, 476],
 216: [1121, 1161, 1125, 1118],
 217: [1043, 1349],
 218: [1121, 546, 536, 1652, 1080],
 219: [1888, 1889, 1480, 1160, 395, 396, 1172, 567, 568, 539, 1887],
 220: [1446, 522, 523, 524, 205, 624, 625, 242],
 221: [1193, 1162, 1197, 188, 1212, 539, 1020],
 222: [580, 582, 583, 150, 537, 538],
 223: [105, 106, 140],
 224: [544, 1227, 1613, 1234, 1210, 542, 543],
 225: [1248, 1249, 103, 395, 1197, 468, 469],
 226: [1540, 1541],
 227: [128, 129],
 228: [1641, 395, 140, 1076, 117, 540, 541],
 229: [546, 970, 1465, 145, 1497],
 230: [1826, 1476, 1195, 525, 1199],
 231: [547, 1227, 1549, 1235, 116, 377, 378],
 232: [466, 189],
 234: [260, 395, 1233, 1492, 565, 566, 1238, 570, 1563, 1564, 1247],
 235: [1604, 1704, 526, 240, 241, 1466],
 236: [1472, 1473, 708, 709, 1577],
 237: [707],
 238: [400, 25, 668, 62],
 239: [1452],
 240: [1864, 1925, 910, 1783],
 241: [39, 942, 472, 473, 1722, 1596],
 242: [545, 421, 905, 914, 247, 1946],
 243: [408, 1211, 360, 1646],
 244: [1264, 1266, 1267, 1268],
 245: [1330, 1326, 1327],
 247: [867, 261, 263, 11, 1041, 831],
 248: [264],
 249: [1219, 1221, 1702, 591, 1043, 314],
 250: [1731,
  1732,
  195,
  6,
  1576,
  1867,
  1069,
  1522,
  403,
  212,
  213,
  404,
  407,
  405,
  534,
  1459,
  406,
  318],
 251: [1323, 1947, 358, 831],
 252: [1862, 1863, 371, 1877, 533, 1913, 1914, 1915, 1916],
 253: [421, 360, 1768, 906, 907, 908, 909, 1043],
 254: [1574, 1769, 230, 992],
 255: [421],
 256: [1763, 1926, 1927, 104, 1928, 426, 1131, 1929, 1933, 1488, 1489],
 257: [141, 1662],
 258: [1703, 830, 1043, 918, 1879, 856, 1917, 1918, 1663],
 259: [596, 1269, 597, 598, 1436],
 260: [510],
 261: [1890, 1480, 1482, 1743, 1018],
 262: [1343],
 264: [1251, 1798, 1255, 1256, 842, 843, 1811],
 265: [1600, 1602, 1356, 1357, 1582, 1078, 1079, 1309],
 266: [1313, 1352, 1257, 1259, 1311],
 267: [1824, 1297, 978, 785, 119, 1951, 1502, 511],
 268: [879, 1200, 1714, 1203, 1204, 1205, 1494, 1207, 1208, 1206, 861, 990],
 269: [882, 994],
 270: [1237, 1077],
 271: [656, 657, 899],
 272: [1856,
  1857,
  1794,
  1858,
  1897,
  1903,
  1489,
  1562,
  915,
  1685,
  1338,
  1339,
  1340],
 273: [880, 913, 1640],
 274: [896, 898],
 275: [883, 1964],
 276: [1075, 1119],
 277: [896, 993, 1699, 1612, 1742, 1775, 1873, 1874, 1021],
 278: [1894, 1000, 1001, 491, 1081, 1082, 478],
 280: [1057, 988],
 281: [79],
 282: [577, 1091, 1579, 78, 593, 594, 244, 1559],
 283: [297, 507, 1145, 1786, 123, 92, 125],
 284: [801, 897, 900, 901, 1510, 1470, 1511, 1509, 185, 1884, 478],
 285: [968, 962, 399],
 286: [1353, 1351],
 287: [1362, 868],
 288: [950, 951, 1115, 1116, 1117],
 289: [952],
 291: [1638, 341, 1690, 601, 602, 603, 605],
 292: [651, 652, 653],
 293: [504, 505, 1163, 503],
 294: [640, 641, 642, 643, 645, 650, 1932, 638, 639],
 295: [929, 772, 773, 1099, 1163, 697, 1053, 767],
 296: [806, 807],
 297: [741,
  1542,
  71,
  614,
  615,
  618,
  619,
  617,
  616,
  620,
  621,
  1648,
  1649,
  1464,
  1791],
 298: [768, 769, 770],
 299: [777, 727],
 300: [1475, 677],
 301: [673, 646, 744, 616, 622, 367, 1136, 666, 636, 637],
 303: [804, 811, 975, 976, 1432],
 304: [647, 663, 1005, 1457, 659, 660, 661, 662, 1014, 665, 666, 348, 349],
 305: [1667, 764, 765, 1014],
 306: [947, 1052],
 307: [762, 1051, 763],
 308: [1302],
 309: [678, 694, 679],
 310: [384, 386, 819, 820, 954, 829],
 311: [1108],
 312: [1148, 278, 1871],
 313: [442],
 314: [771],
 315: [110],
 316: [984, 980],
 317: [832],
 318: [1065, 1173],
 319: [449, 1931, 1262],
 320: [879],
 321: [1584, 1583],
 322: [1944, 1175, 823],
 323: [1096],
 324: [825, 827, 828, 1263],
 326: [648, 649, 956, 957],
 327: [805],
 328: [1721, 178, 179, 1670],
 329: [1016, 1168, 15],
 330: [1608, 1651, 93, 15],
 331: [424, 745, 746, 747, 1553],
 332: [1458, 1186, 1397, 1166],
 335: [552, 1593, 3, 1523],
 336: [429],
 339: [974, 742],
 340: [1844, 1845],
 342: [1737, 1738],
 343: [1035],
 344: [1539],
 345: [1185, 164, 1084],
 346: [706, 1673, 1674, 1675, 1676, 1677],
 347: [1865, 1516, 1518, 1394, 1050, 571],
 348: [1401, 1402, 1782],
 349: [545, 1614, 1393, 532, 1530],
 350: [1483],
 351: [1392, 1547],
 352: [548, 1483, 1784, 1431, 1720, 1401],
 353: [299],
 354: [723, 724, 725, 726, 728],
 355: [535],
 356: [344, 1823],
 357: [512, 513, 514, 515, 516, 517, 1481, 1322, 182, 505],
 358: [1850],
 359: [632, 633, 634, 1526],
 360: [416, 201, 1610, 859, 414, 415],
 361: [866],
 362: [1120, 1606, 1545, 76, 1711, 1815, 1693],
 363: [1153, 331],
 364: [1639],
 365: [329, 330, 778, 1637],
 366: [810, 812],
 367: [350, 1302],
 368: [1642, 26, 943],
 369: [1220, 585, 1456, 1524, 572],
 370: [384, 809, 814, 817, 818, 1586, 1854],
 371: [43],
 372: [1126],
 373: [874],
 374: [1241, 586, 588, 589, 1240, 921],
 375: [1672, 137, 1451, 1556, 1557, 1558, 283, 1788, 1789],
 376: [1650, 1106, 1102],
 377: [972, 853, 471, 1242, 251, 573, 574, 575],
 380: [228],
 381: [411],
 382: [49, 1388, 1389],
 383: [1484],
 385: [1512, 4],
 386: [1189, 45, 1745, 1746, 1142, 926],
 387: [875, 1716, 1045, 1461, 1469],
 388: [1853],
 389: [1828, 1829, 1832, 715, 1631],
 390: [1180, 1773, 1847],
 391: [1366],
 392: [608, 611, 127, 606, 607],
 393: [1280,
  1281,
  704,
  869,
  871,
  1384,
  1866,
  1744,
  1276,
  23,
  1689,
  1658,
  1277,
  1757,
  1278,
  1279],
 394: [1795, 904, 368, 146, 824],
 395: [752, 1500],
 396: [1643, 139, 717, 718, 147, 758, 1146],
 397: [276],
 398: [1094,
  1655,
  1097,
  365,
  798,
  784,
  849,
  848,
  919,
  895,
  852,
  630,
  631,
  1852,
  797,
  894,
  1087],
 399: [1682, 1683],
 400: [1344, 1345, 1346, 1347, 560, 49, 1432],
 401: [758],
 402: [300],
 403: [1447, 1705, 1520, 22, 1758, 63],
 404: [50],
 405: [1151],
 406: [184, 658, 1551],
 408: [114],
 409: [132, 781, 14, 1453, 308, 1719],
 410: [284],
 412: [1421],
 414: [674],
 416: [296, 21],
 417: [72, 1777, 1827, 94],
 418: [986, 1013],
 419: [1272, 721, 722, 591],
 420: [1441,
  737,
  1283,
  612,
  1284,
  739,
  1736,
  1739,
  1740,
  1741,
  1903,
  112,
  1851,
  1503],
 421: [1260, 1605],
 422: [1043],
 423: [155, 1796, 509, 92],
 424: [155, 507],
 425: [1800, 1801, 1802, 1513, 1807],
 426: [1643],
 427: [1416],
 429: [1368, 1766],
 432: [1780, 1781, 1886],
 433: [267, 271, 326, 270],
 434: [54],
 435: [782],
 437: [1093],
 438: [1089, 421, 1482, 1708, 247],
 439: [1229],
 442: [1578, 1922],
 443: [716],
 444: [1941],
 445: [246],
 446: [1113, 953, 1114],
 448: [125],
 449: [865],
 450: [577, 1607, 1355, 1460, 92],
 451: [1201, 1333, 1495],
 452: [1104, 1899, 1898, 1891],
 453: [1232, 1021, 229],
 455: [12],
 456: [696, 1132, 695],
 457: [1963],
 458: [1331, 1292, 1332],
 459: [1074],
 500: [592],
 501: [1861],
 502: [861],
 504: [1869, 1870],
 505: [1883, 915],
 507: [1881, 1882],
 508: [937, 938, 939, 9],
 509: [993, 802, 1878, 863],
 510: [1920, 1921, 1300],
 511: [1848, 1849],
 513: [1393, 1930],
 514: [1705, 579, 590, 183],
 515: [676, 1644, 878, 1936, 1937, 1938, 702, 703],
 516: [1653, 525],
 517: [1222],
 518: [556],
 519: [8, 510],
 520: [119, 999],
 521: [1274],
 523: [1970],
 524: [442, 434, 435],
 525: [1424]}

In [25]:
reporte = 'D:/OneDrive - Ingenio Azucarero Guabira S.A/_DATOS_PYTHON/_tes_pdf/' + str(1) + '.pdf'

In [26]:
props = [1696, 17, 1748]

In [31]:
p = []
p.append(reporte)
for prop in props:
    name = 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_' + str(prop) + '.pdf'
    p.append(name)
p

['D:/OneDrive - Ingenio Azucarero Guabira S.A/_DATOS_PYTHON/_tes_pdf/1.pdf',
 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_1696.pdf',
 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_17.pdf',
 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_1748.pdf']

In [32]:
from PyPDF2 import PdfMerger

In [33]:
fusionador = PdfMerger()

In [48]:
for llave, valor in dict_keys.items():
    fusionador = PdfMerger()
    reporte = 'D:/OneDrive - Ingenio Azucarero Guabira S.A/_DATOS_PYTHON/_tes_pdf/' + str(llave) + '.pdf'
    p = []
    p.append(reporte)
    for prop in valor:
        name = 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_' + str(prop) + '.pdf'
        p.append(name)
    print(p)
    print('-*************************************')
    
    for pdf in p:
        fusionador.append(open(pdf, 'rb'))
    salida_r = 'D:/OneDrive - Ingenio Azucarero Guabira S.A/_DATOS_PYTHON/_tes_pdf_2/'
    with open(salida_r + str(llave) + '.pdf', 'wb') as salida:
        fusionador.write(salida)
    print(llave)

['D:/OneDrive - Ingenio Azucarero Guabira S.A/_DATOS_PYTHON/_tes_pdf/1.pdf', 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_1696.pdf', 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_17.pdf', 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_1748.pdf']
-*************************************
1
['D:/OneDrive - Ingenio Azucarero Guabira S.A/_DATOS_PYTHON/_tes_pdf/2.pdf', 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_265.pdf', 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_369.pdf', 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVA

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/PLANOS_2/PLANO_1008.pdf'

In [34]:
for pdf in p:
    fusionador.append(open(pdf, 'rb'))

In [40]:
salida_r = 'D:/OneDrive - Ingenio Azucarero Guabira S.A/_DATOS_PYTHON/_tes_pdf_2/'

In [41]:
with open(salida_r + str(1) + '.pdf', 'wb') as salida:
    fusionador.write(salida)